In [111]:
import multiprocessing as mp
import jsonpickle
import ctypes

class TimeoutError(RuntimeError):
    pass

class ProcessAsyncCall(object):
    def __init__(self, fnc, callback = None):
        self.Callable = fnc
        self.Callback = callback
        self.Result   = None
        self.Manager  = None

    def __call__(self, *args, **kwargs):
        self.Manager = mp.Manager()
        self.Result  = self.Manager.Value(ctypes.c_wchar,'')
        self.proc = mp.Process(target = self.run, name = self.Callable.__name__, args = args, kwargs = kwargs)
        self.proc.start()
        return self

    def wait(self, timeout = None):
        self.proc.join(timeout)
        if self.proc.is_alive():
            raise TimeoutError()
        else:
            return self.Result

    def get(self):
        if self.Result == None:
            self.wait()
                        
        return jsonpickle.decode(self.Result.value)

    def run(self, *args, **kwargs):
        try:
            result = self.Callable(*args, **kwargs)
            try: 
                packed_result = jsonpickle.encode(result)
                self.Result.value = "%s" % packed_result
            except Exception as e:
                print(e)
                raise e
                
            if self.Callback:
                self.Callback(self.Result)
        except Exception as e:
            print(e, args, kwargs)
            raise e

class ThreadAsyncCall(object):
    def __init__(self, fnc, callback = None):
        self.Callable = fnc
        self.Callback = callback
        self.Result = None
        
    def __call__(self, *args, **kwargs):
        self.Thread = threading.Thread(target = self.run, name = self.Callable.__name__, args = args, kwargs = kwargs)
        self.Thread.start()
        return self

    def wait(self, timeout = None):
        self.Thread.join(timeout)
        
        if self.Thread.isAlive():
            raise TimeoutError()
        else:
            return self.Result

    def get(self):
        if self.Result == None:
            self.wait()
        return self.Result

    def run(self, *args, **kwargs):
        try:
            self.Result = self.Callable(*args, **kwargs)
            if self.Callback:
                self.Callback(self.Result)
        except Exception as e:
            print(e, args, kwargs)
            traceback.print_exc(file=sys.stdout)

class ThreadAsyncMethod(object):
    def __init__(self, fnc, callback=None):
        self.Callable = fnc 
        self.Callback = callback

    def __call__(self, *args, **kwargs):
        return ThreadAsyncCall(self.Callable, self.Callback)(*args, **kwargs)

class ProcessAsyncMethod(object):
    def __init__(self, fnc, callback=None):
        self.Callable = fnc 
        self.Callback = callback

    def __call__(self, *args, **kwargs):
        return ProcessAsyncCall(self.Callable, self.Callback)(*args, **kwargs)

def ProcessAsync(fnc = None, callback = None):
    if fnc == None:
        def AddAsyncCallback(fnc):
            return ProcessAsyncMethod(fnc, callback)
        return AddAsyncCallback
    else:
        return ProcessAsyncMethod(fnc, callback)

def Async(fnc = None, callback = None):
    if fnc == None:
        def AddAsyncCallback(fnc):
            return ThreadAsyncMethod(fnc, callback)
        return AddAsyncCallback
    else:
        return ThreadAsyncMethod(fnc, callback)


In [121]:
@ProcessAsync
def myAsyncCall(arg1, arg2):
    import time
    print("myAsynCall working")
    
    m ={}
    m[arg1] = arg2
    
    return m

a = myAsyncCall(10,20)

myAsynCall working


In [122]:
a.wait()

<ValueProxy object, typeid 'Value' at 0x2afcb6cca7b8>

In [123]:
print(a.get())

{'10': 20}
